# Chapter 2 : Modeling in the Frequency Domain (Part 3)
---

In [1]:
import sys

if (path := "C:/Users/Tom/pycharm-projects/python-control") not in sys.path:
    sys.path.append(path)

import sympy as sp

from IPython.display import display

from python_control import Quantity, TransferFunction
from python_control.modeling.mechanical import (
    Spring, Damper, Mass,
    TorsionSpring, TorsionDamper, Inertia,
    GearRatio
)

Q_ = Quantity

## 2.5 : Translational Mechanical System Transfer Functions

### Example 2.16 : Transfer Function - One Equation of Motion

Find the transfer function $X(s) / F(s)$ for the system in the figure below.

![mass-spring-damper system](./images/example_2-16a.png)

Free-body diagram: Place all the forces on the mass felt by the mass. We assume that the mass is travelling in the direction of the applied force $f(t)$. All other forces impede the motion to the right and act to oppose it. Hence, the spring, viscous damper, and the mass inertia point to the left.

![free body diagram](./images/example_2-16b.png)

Differential equation of motion:
$$
M\frac{{{d^2}x\left( t \right)}}{{dt}} + {f_v}\frac{{dx\left( t \right)}}{{dt}} + Kx\left( t \right) = f\left( t \right)
$$

Laplace transform:
$$M{s^2} \cdot X\left( s \right) + {f_v}s \cdot X\left( s \right) + K \cdot X\left( s \right) = F\left( s \right)$$
$$\left( {M{s^2} + {f_v}s + K} \right) \cdot X\left( s \right) = F\left( s \right)$$

Transfer function:
$$G\left( s \right) = \frac{{X\left( s \right)}}{{F\left( s \right)}} = \frac{1}{{M{s^2} + fs + K}}$$

For each mechanical compononent we can define an impedance $X(s) / F(s)$, which is essentially a transfer function between the output $X(s)$, being the displacement of the mass-spring-damper system, and the input $F(s)$, being the external force applied to the system:

In [2]:
mass = Mass('M')
mass.Z.expr

1.0*M*s**2

In [3]:
damper = Damper('f_v')
damper.Z.expr

1.0*f_v*s

In [4]:
spring = Spring('K')
spring.Z.expr

1.0*K

The total impedance of the system is the sum of the impedances of mass, damper, and spring:

In [5]:
Z = mass.Z + damper.Z + spring.Z
Z.expr

1.0*K + 1.0*M*s**2 + 1.0*f_v*s

In [6]:
type(Z)

python_control.core.transfer_function.TransferFunction

### Example 2.17 : Transfer Function - Two Degrees of Freedom

Find the transfer function $X_2(s) / F(s)$ for the system in the figure below.

![two-degrees-of-freedom translational mechanical system](./images/example_2-17.png)

Mechanical components that make up the system:

In [7]:
mass1 = Mass('M1')
spring1 = Spring('K1')
damper1 = Damper('f_v1')
damper3 = Damper('f_v3')
spring2 = Spring('K2')
mass2 = Mass('M2')
damper2 = Damper('f_v2')
spring3 = Spring('K3')

**Motion of $x_1$**

Sum of impedances connected to the motion at $x_1$:

In [8]:
Z11 = mass1.Z + spring1.Z + damper1.Z + damper3.Z + spring2.Z

Sum of impedances between $x_1$ and $x_2$: 

In [9]:
Z12 = damper3.Z + spring2.Z

**Motion of $x_2$**

Sum of impedances connected to the motion at $x_2$:

In [10]:
Z22 = mass2.Z + damper3.Z + spring2.Z + damper2.Z + spring3.Z

Sum of impedances between $x_2$ and $x_1$: 

In [11]:
Z21 = damper3.Z + spring2.Z

**Transfer Function**

The two equations of motion of this system can be written in matrix notation like:
$$
\left[ {\begin{array}{*{20}{c}}
{{Z_{11}}}&{ - {Z_{12}}}\\
{ - {Z_{21}}}&{{Z_{22}}}
\end{array}} \right] \cdot \left[ {\begin{array}{*{20}{c}}
{{X_1}}\\
{{X_2}}
\end{array}} \right] = \left[ {\begin{array}{*{20}{c}}
{{F_1}}\\
{{F_2}}
\end{array}} \right]
$$
To find the transfer function $X_2(s) / F(s)$, this set of two equations can first be solved symbolically with *Sympy*. Note that no external force is applied at $x_2$.

In [12]:
F1 = sp.Symbol('F')
F2 = 0

Z = sp.Matrix([[Z11.expr, -Z12.expr], [-Z21.expr, Z22.expr]])
F = sp.Matrix([[F1], [F2]])

X = Z.LUsolve(F)
X2 = X[1]
X2

-F*(-1.0*K2 - 1.0*f_v3*s)/((1.0*K1 + 1.0*K2 + 1.0*M1*s**2 + 1.0*s*(1.0*f_v1 + 1.0*f_v3))*(1.0*K2 + 1.0*K3 + 1.0*M2*s**2 + 1.0*s*(1.0*f_v2 + 1.0*f_v3) - (-1.0*K2 - 1.0*f_v3*s)**2/(1.0*K1 + 1.0*K2 + 1.0*M1*s**2 + 1.0*s*(1.0*f_v1 + 1.0*f_v3))))

In [13]:
G = TransferFunction(X2 / F1)
G.expr

(K2/(M1*M2) + f_v3*s/(M1*M2))/(s**4 + s**3*(M1*f_v2 + M1*f_v3 + M2*f_v1 + M2*f_v3)/(M1*M2) + s**2*(K1*M2 + K2*M1 + K2*M2 + K3*M1 + f_v1*f_v2 + f_v1*f_v3 + f_v2*f_v3)/(M1*M2) + s*(K1*f_v2 + K1*f_v3 + K2*f_v1 + K2*f_v2 + K3*f_v1 + K3*f_v3)/(M1*M2) + (K1*K2 + K1*K3 + K2*K3)/(M1*M2))

### Example 2.18 : Equations of Motion by Inspection

Write, but do not solve, the equations of motion for the mechanical network in the figure below.

![three-degrees-of-freedom translational mechanical system](./images/example_2-18.png)

Mechanical components that make up the system:

In [14]:
mass1 = Mass('M1')
spring1 = Spring('K1')
damper1 = Damper('f_v1')
spring2 = Spring('K2')
damper3 = Damper('f_v3')
mass2 = Mass('M2')
damper2 = Damper('f_v2')
damper4 = Damper('f_v4')
mass3 = Mass('M3')

**Motion of $x_1$**

Sum of impedances connected to the motion at $x_1$:

In [15]:
Z11 = mass1.Z + spring1.Z + damper1.Z + damper3.Z + spring2.Z

Sum of impedances between $x_1$ and $x_2$:

In [16]:
Z12 = spring2.Z

Sum of impedances between $x_1$ and $x_3$:

In [17]:
Z13 = damper3.Z

**Motion of $x_2$**

Sum of impedances connected to the motion at $x_2$:

In [18]:
Z22 = mass2.Z + spring2.Z + damper2.Z + damper4.Z

Sum of impedances between $x_2$ and $x_1$:

In [19]:
Z21 = Z12

Sum of impedances between $x_2$ and $x_3$:

In [20]:
Z23 = damper4.Z

**Motion of $x_3$**

Sum of impedances connected to the motion at $x_3$:

In [21]:
Z33 = mass3.Z + damper3.Z + damper4.Z

Sum of impedances between $x_3$ and $x_1$:

In [22]:
Z31 = Z13

Sum of impedances between $x_3$ and $x_2$:

In [23]:
Z32 = Z23

**Equations of Motion**

The three equations of motion of this system can be written in matrix notation like:
$$
\left[ {\begin{array}{*{20}{c}}
{{Z_{11}}}&{ - {Z_{12}}}&{ - {Z_{13}}}\\
{ - {Z_{21}}}&{{Z_{22}}}&{ - {Z_{23}}}\\
{ - {Z_{31}}}&{ - {Z_{32}}}&{{Z_{33}}}
\end{array}} \right] \cdot \left[ {\begin{array}{*{20}{c}}
{{X_1}}\\
{{X_2}}\\
{{X_3}}
\end{array}} \right] = \left[ {\begin{array}{*{20}{c}}
{{F_1}}\\
{{F_2}}\\
{{F_3}}
\end{array}} \right]
$$

In [24]:
F1, F2, F3 = 0, sp.Symbol('F'), 0
X1, X2, X3 = sp.symbols(['X1', 'X2', 'X3'])

Z = sp.Matrix([
    [Z11.expr, -Z12.expr, -Z13.expr], 
    [-Z21.expr, Z22.expr, -Z23.expr],
    [-Z31.expr, -Z32.expr, Z33.expr]
])
X = sp.Matrix([[X1], [X2], [X3]])
F = sp.Matrix([[F1], [F2], [F3]])

for eq in Z * X - F:
    display(eq)

-1.0*K2*X2 + X1*(1.0*K1 + 1.0*K2 + 1.0*M1*s**2 + 1.0*s*(1.0*f_v1 + 1.0*f_v3)) - 1.0*X3*f_v3*s

-F - 1.0*K2*X1 + X2*(1.0*K2 + 1.0*M2*s**2 + 1.0*s*(1.0*f_v2 + 1.0*f_v4)) - 1.0*X3*f_v4*s

-1.0*X1*f_v3*s - 1.0*X2*f_v4*s + X3*(1.0*M3*s**2 + 1.0*s*(1.0*f_v3 + 1.0*f_v4))

### Skill-Assessment Exercise 2.8

Find the transfer function $G(s) = X_2(s) / F(s)$ for the translational mechanical system shown in the figure below.

![translational mechanical system for skill-assessement 2.8](./images/skill_exercise_2-8.png)

Mechanical components that make up the system:

In [25]:
mass1 = Mass(Q_(1, 'kg'))
damper1 = Damper(Q_(1, 'N * s / m'))
damper2 = Damper(Q_(1, 'N * s / m'))
damper3 = Damper(Q_(1, 'N * s / m'))
spring = Spring(Q_(1, 'N / m'))
mass2 = Mass(Q_(1, 'kg'))
damper4 = Damper(Q_(1, 'N * s / m'))

**Motion of $x_1$**

Sum of impedances connected to the motion at $x_1$:

In [26]:
Z11 = mass1.Z + damper1.Z + damper2.Z + spring.Z + damper3.Z

Sum of impedances between $x_1$ and $x_2$:

In [27]:
Z12 = damper1.Z + damper2.Z + spring.Z + damper3.Z

**Motion of $x_2$**

Sum of impedances connected to the motion at $x_2$:

In [28]:
Z22 = mass2.Z + damper1.Z + damper2.Z + spring.Z + damper3.Z + damper4.Z

Sum of impedances between $x_2$ and $x_1$:

In [29]:
Z21 = Z12

**Transfer Function**

In [30]:
F1 = sp.Symbol('F')
F2 = 0

Z = sp.Matrix([[Z11.expr, -Z12.expr], [-Z21.expr, Z22.expr]])
F = sp.Matrix([[F1], [F2]])

X = Z.LUsolve(F)
X2 = X[1]
X2

G = TransferFunction(X2 / F1)
G.expr

(3.0*s + 1.0)/(1.0*s**4 + 7.0*s**3 + 5.0*s**2 + 1.0*s)

## 2.6 : Rotational Mechanical System Transfer Functions

### Example 2.19 : Transfer Function - Two Equations of Motion

Find the transfer function $\theta_2(s) / T(s)$. The rod is supported by bearings at either end and is undergoing torsion. A torque is applied at the left, and the displacement is measured at the right.

![physical system](./images/example_2-19.png)

We assume that the torsion acts like a spring concentrated at one particular point in the rod, with an inertia $J_1$ to the left and an inertia $J_2$ to the right. There are two degrees of freedom, since each inertia can be rotated while the other is held still.

Mechanical components that make up the system:

In [31]:
damper1 = TorsionDamper('D1')
inertia1 = Inertia('J1')
spring = Spring('K')
inertia2 = Inertia('J2')
damper2 = TorsionDamper('D2')

**Rotation $\theta_1$**

Sum of impedances connected to the rotation at $\theta_1$:

In [32]:
Z11 = damper1.Z + inertia1.Z + spring.Z

Sum of impedances connected between $\theta_1$ and $\theta_2$:

In [33]:
Z12 = spring.Z

**Rotation $\theta_2$**

Sum of impedances connected to the rotation at $\theta_2$:

In [34]:
Z22 = spring.Z + inertia2.Z + damper2.Z

Sum of impedances connected between $\theta_2$ and $\theta_1$:

In [35]:
Z21 = Z12

**Transfer Function**

In [36]:
Z = sp.Matrix([[Z11.expr, -Z12.expr], [-Z21.expr, Z22.expr]])

T1 = sp.Symbol('T1')
T2 = 0
T = sp.Matrix([[T1], [T2]])

theta = Z.LUsolve(T)
theta

Matrix([
[(1.0*K**2*T1/((1.0*D1*s + 1.0*J1*s**2 + 1.0*K)*(1.0*D2*s + 1.0*J2*s**2 - 1.0*K**2/(1.0*D1*s + 1.0*J1*s**2 + 1.0*K) + 1.0*K)) + T1)/(1.0*D1*s + 1.0*J1*s**2 + 1.0*K)],
[                                           1.0*K*T1/((1.0*D1*s + 1.0*J1*s**2 + 1.0*K)*(1.0*D2*s + 1.0*J2*s**2 - 1.0*K**2/(1.0*D1*s + 1.0*J1*s**2 + 1.0*K) + 1.0*K))]])

In [37]:
theta2 = theta[1]
G = TransferFunction(theta2 / T1)
G.expr

K/(J1*J2*(s**4 + s**3*(D1*J2 + D2*J1)/(J1*J2) + s**2*(D1*D2 + J1*K + J2*K)/(J1*J2) + s*(D1*K + D2*K)/(J1*J2)))

### Skill-Assessment Exercise 2.9

Find the transfer function $G(s) = \theta_2(s) / T(s)$ for the rotational system in the figure below.

![rotational mechanical system](./images/skill_exercise_2-9.png)

Mechanical components that make up the system:

In [38]:
inertia1 = Inertia(Q_(1, 'kg * m**2'))
spring1 = TorsionSpring(Q_(1, 'N * m / rad'))
damper1 = TorsionDamper(Q_(1, 'N * m * s / rad'))
spring2 = TorsionSpring(Q_(1, 'N * m / rad'))
damper2 = TorsionDamper(Q_(1, 'N * m * s / rad'))

**Rotation $\theta_1$**

Sum of impedances connected to the rotation at $\theta_1$:

In [39]:
Z11 = inertia1.Z + spring1.Z + damper1.Z

Sum of impedances connected between $\theta_1$ and $\theta_2$:

In [40]:
Z12 = spring1.Z + damper1.Z

**Rotation $\theta_2$**

Sum of impedances connected to the rotation at $\theta_2$:

In [41]:
Z22 = spring1.Z + damper1.Z + spring2.Z + damper2.Z

Sum of impedances connected between $\theta_2$ and $\theta_1$:

In [42]:
Z21 = Z12

**Transfer Function**

In [43]:
Z = sp.Matrix([[Z11.expr, -Z12.expr], [-Z21.expr, Z22.expr]])

T1 = sp.Symbol('T1')
T2 = 0
T = sp.Matrix([[T1], [T2]])

theta = Z.LUsolve(T)
theta

Matrix([
[(T1*(-1.0*s - 1.0)**2/((2.0*s - (-1.0*s - 1.0)**2/(1.0*s**2 + 1.0*s + 1.0) + 2.0)*(1.0*s**2 + 1.0*s + 1.0)) + T1)/(1.0*s**2 + 1.0*s + 1.0)],
[                                  -T1*(-1.0*s - 1.0)/((2.0*s - (-1.0*s - 1.0)**2/(1.0*s**2 + 1.0*s + 1.0) + 2.0)*(1.0*s**2 + 1.0*s + 1.0))]])

In [44]:
theta2 = theta[1]
G = TransferFunction(theta2 / T1)
G.expr

0.5/(1.0*s**2 + 0.5*s + 0.5)

## 2.7 : Transfer Functions for Systems with Gears

### Example 2.21 : Transfer Function - System with Lossless Gears

Find the transfer function $\theta_2(s) / T_1(s)$ for the system in the figure below.

![rotational mechanical system with gears](./images/example_2-21.png)

Mechanical impedances in the system:

In [45]:
Z_J1 = Inertia('J1').Z
Z_D1 = TorsionDamper('D1').Z
Z_J2 = Inertia('J2').Z
Z_D2 = TorsionDamper('D2').Z
Z_K = TorsionSpring('K2').Z

Torque $T_1$ about the input shaft of the gearbox:

In [46]:
T1 = sp.Symbol('T1')

Note that the inertias $J_1$ and $J_2$ do not undergo linearly independent motion, since they are tied together by the gears. Thus, there is only one degree of freedom and hence one equation of motion.

The impedances $J_1$ and $D_1$ and the torque $T_1$ can be reflected from the input to the output shaft with the `GearRatio` class. Its method `reflect_to_output()` reflects impedances and torques on the input side to the output side of the gearbox.

In [47]:
gearbox = GearRatio(N_in='N1', N_out='N2')

Z_J1to2 = gearbox.reflect_to_output(Z_in=Z_J1)
Z_D1to2 = gearbox.reflect_to_output(Z_in=Z_D1)

T1to2 = gearbox.reflect_to_output(T_in=T1)

**Total impedance on the output shaft**

In [48]:
Z = Z_J2 + Z_D2 + Z_K
Z += Z_J1to2 + Z_D1to2

**Equation of motion reflected to the output shaft**

In [49]:
theta2 = sp.Symbol('theta_2')
eom = Z.expr * theta2 - T1to2
eom

theta_2*(K2 + s**2*(J1*N2**2 + J2*N1**2)/N1**2 + s*(D1*N2**2 + D2*N1**2)/N1**2) - N2*T1/N1

Solve the equation of motion for $\theta_2$:

In [50]:
theta2 = sp.solve(eom, theta2)[0]
theta2

N1*N2*T1/(D1*N2**2*s + D2*N1**2*s + J1*N2**2*s**2 + J2*N1**2*s**2 + K2*N1**2)

**Transfer function** $\theta_2(s) / T_1(s)$:

In [51]:
G = TransferFunction(theta2 / T1)
G.expr

N1*N2/((J1*N2**2 + J2*N1**2)*(K2*N1**2/(J1*N2**2 + J2*N1**2) + s**2 + s*(D1*N2**2 + D2*N1**2)/(J1*N2**2 + J2*N1**2)))

### Example 2.22 :  Transfer Function - Gears with Loss

Find the transfer function $\theta_1(s) / T_1(s)$ for the system in the figure below.

![system using a gear train](./images/example_2-22.png)

This system does not have lossless gears. All of the gears have inertia, and for some shafts there is viscous friction. To solve the problem, we want to reflect all of the impedances to the input shaft.

The system has two gear ratios, $N_1/N_2$ and $N_3/N_4$:

In [52]:
gear_ratio1 = GearRatio(N_in='N1', N_out='N2')
gear_ratio2 = GearRatio(N_in='N3', N_out='N4')

These two gear ratios can be combined into one overall gear ratio between the input shaft of the first gear ratio and the output shaft of the second gear ratio:

In [53]:
gear_ratio = gear_ratio1 + gear_ratio2

Mechanical impedances on the input shaft of the first gear ratio:

In [54]:
Z_J1 = Inertia('J1').Z
Z_D1 = TorsionDamper('D1').Z

Reflecting impedances on the output shaft of the first gear ratio to the input shaft of the first gear ratio: 

In [55]:
Z_J2 = Inertia('J2').Z
Z_D2 = TorsionDamper('D2').Z
Z_J3 = Inertia('J3').Z

Z_J2to1 = gear_ratio1.reflect_to_input(Z_out=Z_J2)
Z_D2to1 = gear_ratio1.reflect_to_input(Z_out=Z_D2)
Z_J3to1 = gear_ratio1.reflect_to_input(Z_out=Z_J3)

Reflecting impedances on the output shaft of the second gear ratio to the input shaft of the first gear ratio:

In [56]:
Z_J4 = Inertia('J4').Z
Z_J5 = Inertia('J5').Z

Z_J4to1 = gear_ratio.reflect_to_input(Z_out=Z_J4)
Z_J5to1 = gear_ratio.reflect_to_input(Z_out=Z_J5)

**Total impedance on the input shaft**

In [57]:
Z_J = Z_J1 + Z_J2to1 + Z_J3to1 + Z_J4to1 + Z_J5to1
Z_J.expr

s**2*(J1*N2**2*N4**2 + J2*N1**2*N4**2 + J3*N1**2*N4**2 + J4*N1**2*N3**2 + J5*N1**2*N3**2)/(N2**2*N4**2)

In [58]:
Z_D = Z_D1 + Z_D2to1
Z_D.expr

s*(D1*N2**2 + D2*N1**2)/N2**2

In [59]:
Z = Z_J + Z_D

**Equation of motion reflected on the input shaft**

In [60]:
theta1 = sp.Symbol('theta_1')
T1 = sp.Symbol('T1')

eom = Z.expr * theta1 - T1

Solve the equation of motion for $\theta_1$:

In [61]:
theta1 = sp.solve(eom, theta1)[0]
theta1

N2**2*N4**2*T1/(s*(D1*N2**2*N4**2 + D2*N1**2*N4**2 + J1*N2**2*N4**2*s + J2*N1**2*N4**2*s + J3*N1**2*N4**2*s + J4*N1**2*N3**2*s + J5*N1**2*N3**2*s))

**Transfer function** $\theta_1(s) / T_1(s)$

In [62]:
G = TransferFunction(theta1 / T1)
G.expr

N2**2*N4**2/((s**2 + s*(D1*N2**2*N4**2 + D2*N1**2*N4**2)/(J1*N2**2*N4**2 + J2*N1**2*N4**2 + J3*N1**2*N4**2 + J4*N1**2*N3**2 + J5*N1**2*N3**2))*(J1*N2**2*N4**2 + J2*N1**2*N4**2 + J3*N1**2*N4**2 + J4*N1**2*N3**2 + J5*N1**2*N3**2))

### Skill-Assessment Exercise 2.10

Find the transfer function $G(s) = \theta_2(s) / T(s)$ for the rotational mechanical system with gears in the figure below.

![rotational mechanical system with gears](./images/skill_exercise_2-10.png)

Define the gear ratio:

In [63]:
gear_ratio = GearRatio(N_in=25, N_out=50)

Mechanical impedances in the system:

In [64]:
Z_K = TorsionSpring(Q_(4, 'N * m / rad')).Z
Z_D = TorsionDamper(Q_(1, 'N * m * s / rad')).Z
Z_J = Inertia(Q_(1, 'kg * m ** 2')).Z

Reflect the impedance of the spring and the rotation angle of the output shaft to the input shaft: 

In [65]:
Z_K_to_1 = gear_ratio.reflect_to_input(Z_out=Z_K)

theta2 = sp.Symbol('theta_2')
theta2_to_1 = gear_ratio.reflect_to_input(theta_out=theta2)

**Overall impedance reflected to the input shaft**

In [66]:
Z = Z_J + Z_D + Z_K_to_1
Z.expr

1.0*s**2 + 1.0*s + 1.0

**Equation of motion reflected to the input shaft**

In [67]:
T = sp.Symbol('T')
eom = Z.expr * theta2_to_1 - T

theta2 = sp.solve(eom, theta2)[0]
theta2

0.5*T/(s**2 + s + 1.0)

**Transfer function** $\theta_2(s) / T(s)$

In [68]:
G = TransferFunction(theta2 / T)
G.expr

0.5/(1.0*s**2 + 1.0*s + 1.0)